# Ověřování hypotézy pomoci $\chi^2$ testu

S 95 % jistotou ověřujeme následující hypotézu: 

*Pokud viník nehody byl pod silným vlivem alkoholu, došlo častěji k těžkým zdravotním
následkům.*

## Příprava knihoven a nahrání dat

Nejprve si nahrajeme potřebné knihovny. V tomto testu si vystačíme s knihovnami pro zpracování dat `numpy` a `pandas` a s knihovnou pro statistickou analýzu `scipy.stats`.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

Nyní si vytvoříme dataframe, do kterého nahrajeme předzpracovaná data ze souboru `accidents.plk.gz`.

In [2]:
df = pd.read_pickle("accidents.pkl.gz", compression='gzip')
df.head()   # zobrazeni 5 prvnich zaznamu

,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,l,n,o,p,q,r,s,t,p5a,region
0,002100160001,4,,2016-01-01,5,0055,1,1,0,2,...,,711403,,Souhlasnýsesměremúseku,Pomalý,554782,451622,GN_V0.1UIR-ADR_410,1,PHA
1,002100160002,4,,2016-01-01,5,0130,1,3,0,2,...,,,,,,,,,1,PHA
2,002100160003,5,,2016-01-01,5,0100,1,2,0,2,...,,,,,,,,,1,PHA
3,002100160004,6,,2016-01-01,5,0120,9,0,0,2,...,,,,,,,,,1,PHA
4,002100160005,6,,2016-01-01,5,2560,2,0,0,2,...,,,,,,,,,1,PHA


## Výběr a zisk dat pro následující testování

Nyní si z načteného datasetu vybereme potřebné sloupce a relevantní záznamy.

Vybereme tedy následující sloupce:

- `p11` - alkohol u nehody viníka přítomem
- `p13a` - usmrceno osob
- `p13b` - těžce zraněno osob

Z datasetu dále vyloučíme záznamy, kdy byl viník nehody pod vlivem drog (`p11`* == 4* a `p11`* == 5*).

In [3]:
df = df[['p11', 'p13a', 'p13b']]
df = df[~df['p11'].isin([4, 5])]
df.head()   # zobrazeni 5 prvnich zaznamu

,p11,p13a,p13b
0,2,0,0
1,2,0,0
2,2,0,0
3,9,0,0
4,0,0,0


## $\chi^2$ test

Nyní zjistíme, zdali je korelace mezi silným vlivem alkoholu u viníka nehody a těžkými zdravotními následky nehody *statisticky významná*, k čemuž využijeme **$\chi^2$ chí-kvadrát testu**.

Předpokládáme, že mezi vstupními daty není korelace (nulová hypotéza *$H_0$*).

Nejprve si *binarizujeme* potřebné proměnné. Do dataframu `df` přidáme následující sloupce:

- `alcohol` - reprezentuje, zdali byl viník nehody pod silným vlivem alkoholu (`True` - u viníka nehody bylo naměřeno 0,8 ‰ nebo více alkoholu v krvi, `False` - jinak)
- `injuries` - reprezentuje, zdali došlo k těžkým zdravotním následkům (`True` - úmrtí nebo těžká zranění, `False` - jinak)

In [4]:
df['alcohol'] = (df['p11'] >= 7)
df['injuries'] = ((df['p13a'] + df['p13b']) > 0)
df.head()   # zobrazeni 5 prvnich zaznamu

,p11,p13a,p13b,alcohol,injuries
0,2,0,0,False,False
1,2,0,0,False,False
2,2,0,0,False,False
3,9,0,0,True,False
4,0,0,0,False,False


Nyní si vytvoříme kontingenční tabulku pomocí funkce `crosstab` z knihovny `pandas` pro všechny 4 kombinace vstupů:

| Alcohol | Injuries |
| ------- | -------- |
| `False` | `False` |
| `False` | `True` |
| `True` | `False` |
| `True` | `True` |

In [5]:
ct = pd.crosstab(df['alcohol'], df['injuries'])
ct  # zobrazeni tabulky

injuries,False,True
alcohol,,
False,457528,10777
True,16492,886


Nyní si můžeme pro získanou tabulku vypočítat **$\chi^2$ chí-kvadrát test**.

Hodnotu $\chi^2_{test}$ lze spočítat pomocí vzorce:

$$\chi^2_{test}=\sum_{\forall i}\frac{(O_i-E_i)^2}{E_i}$$

- $O_i$ - získaná hodnota
- $E_i$ - očekávaná hodnota, kterou lze vypočítat pomocí vzorce:

$$E_i=\frac{\sum_{r \in row(i)}{O_r} \cdot \sum_{c \in col(i)}{O_c}}{\sum_{\forall j}{O_j}}$$

Dále je možné vypočítat pravděpodobnost  $P(\chi^2 > \chi^2_{test})$, kterou je možné určit díky následující funkci hustoty rozložení pravděpodobnosti:

$$f(x, DF)=\frac{1}{2^{DF/2-1} \Gamma \left( DF/2 \right)} x^{DF-1} e^{ -x^2/2}$$

- $DF$ - stupeň volnosti

Všechny výše zmíněné úkony můžeme ale nechat na funkci `scipy.stats.chi2_contingency()`(kterou také použijeme). Ta vypočítá *chí-kvadrát* statistiku (**xi2**), *p-hodnotu*(tedy danou pravděpodobnost - **p-value**) a dále k s těmito hodnotami vrací i stupeň volnosti (**DF**) a očekávané hodnoty (**E**). Funkce vrací hodnoty ve formátu 4-tice (**xi2**, **p-value**, **DF**, **E**), přičemž k určení, zdali je korelace statisticky významná potřebujeme znát *p-hodnotu*(**p-value**).

In [6]:
xi2, p, _, _ = st.chi2_contingency(ct)
display(f'P(X^2 > {xi2:.2f}) = {p}')    # zobrazeni vysledne pravdepodobnosti

'P(X^2 > 558.17) = 2.09715057003383e-123'

Hodnota $2.1\cdot10^{-123}$ (*p-hodnota*) je mnohem menší než $0.05$, z toho důvodu zamítneme nulovou hypotézu ($H_0$ - mezi vstupními daty není korelace) a můžeme tedy říct, že korelace mezi silným vlivem alkoholu u viníka a těžkými zdravotními následky nehody je *statisticky významná*.

Víme tedy, že mezi vstupními daty je korelace, ovšem zatím nejsme schopni říci, zdali pod silným vlivem alkoholu došlo opravdu častěji k těžkým zdravotním následkům. Z dat v kontingenční tabulce `ct` jsme ale schopni si vypočítat pravděpodobnosti těžkých zdravotních následů, pokud byl viník pod silným vlivem alkoholu, či nikoli.

In [7]:
p_alcohol = ct[1][1]/(ct[0][1] + ct[1][1])
p_not_alcohol = ct[1][0]/(ct[0][0] + ct[1][0])
display(f"p_alcohol = {p_alcohol:.4f}")
display(f"p_not_alcohol = {p_not_alcohol:.4f}")

'p_alcohol = 0.0510'

'p_not_alcohol = 0.0230'

Zde vidíme, že pod silným vlivem alkoholu došlo v 5 % případů k těžkým zdravotním následům, což je více než dvojnásobná pravděpodobnost oproti tomu, kdy řidič pod silným vlivem alkoholu nebyl. Tedy můžeme s jistotou říct, že pokud byl viník pod silným vlivem alkoholu, tak došlo častěji k těžkým zdravotním následkům (to, že je tato korelace *statisticky význámná významná* už víme díky **$\chi^2$ chí-kvadrát testu**).

Ke zjištění korelace můžeme také použít **Pearsonův korelační koeficient** dle vzorce:

$$r_{xy} =\frac{\sum ^n _{i=1}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum ^n _{i=1}(x_i - \bar{x})^2} \sqrt{\sum ^n _{i=1}(y_i - \bar{y})^2}}$$

Celý výpočet můžeme nechat na funkci `scipy.stats.pearsonr()`. Tato funkce vrací *Pearsonův korelační koeficient* (**corr_coef**) a *p-hodnotu* (**p-value**) jako dvojici (**corr_coef**, **p-value**). *P-hodnota* nás zde nyní ale nezajímá, jelikož *statistickou významnost* korelace jsme již ověřili pomocí **$\chi^2$ chí-kvadrát testu**.

In [8]:
corr_coef, _ = st.stats.pearsonr(df['alcohol'], df['injuries'])
display(f"corr_coef = {corr_coef}")  # Pearsonuv korelacni koeficient

'corr_coef = 0.033936908140830946'

Zde nám vychází pozitivní **Pearsonův korelační koeficient**, což znamená, že pod silným vlivem alkoholu u viníka nehody došlo častěji k těžkým zdravotním následkům (Pokud by byl korelační koeficient záporný, znamenalo by to, že při silném vlivu alkoholu u viníka, došlo k těžkým zdravotním následkům méně často, než když viník pod silným vlivem alkoholu nebyl).

## Inspirace

Celý tento notebook vychází z přednášek předmětu IZV a notebooku 01_korelace.ipynb dostupného v souborech k předmětu.